In [1]:
with open('input.txt', 'r') as input_file:
    text = input_file.read()

In [2]:
unique_chars = sorted(list(set(text)))
vocab_size = len(unique_chars)
print(''.join(unique_chars))
print(len(unique_chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
ctoi = { char:ind for ind,char in enumerate(unique_chars)}
itoc = {ind:char for ind,char in enumerate(unique_chars)}

encode = lambda s: [ctoi[z] for z in s]
decode = lambda i: ''.join(itoc[z] for z in i)

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [50]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
data = torch.tensor(encode(text), dtype = torch.long)
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [6]:
# train and test split. we are considering 90% of the data as our training data.

factor = int(0.9 * len(data))
train = data[:factor]
val = data[factor:]

In [7]:
blk = 8
print(train[:blk],'\n')

def get_targets(mat, tar, block_size):
    for _ in range(block_size):
        context = mat[:_+1]
        target = tar[_]
        print(f'context:{context}',f'target: {target}\n')


tensor([18, 47, 56, 57, 58,  1, 15, 47]) 



In [8]:
torch.manual_seed(1337)
batch_size = 4
blk_size = 8

def get_batch(split):
    data = train if split == 'train' else val
    indices = torch.randint((len(data) - blk_size), (batch_size,))
    x = torch.stack([data[i:i+blk_size] for i in indices])
    y = torch.stack([data[i+1:i+blk_size+1] for i in indices])
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')

x.shape,y.shape,x,y

(torch.Size([4, 8]),
 torch.Size([4, 8]),
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))

In [9]:
for _ in range(batch_size):
    get_targets(x[_],y[_],len(x[_]))
    

context:tensor([24]) target: 43

context:tensor([24, 43]) target: 58

context:tensor([24, 43, 58]) target: 5

context:tensor([24, 43, 58,  5]) target: 57

context:tensor([24, 43, 58,  5, 57]) target: 1

context:tensor([24, 43, 58,  5, 57,  1]) target: 46

context:tensor([24, 43, 58,  5, 57,  1, 46]) target: 43

context:tensor([24, 43, 58,  5, 57,  1, 46, 43]) target: 39

context:tensor([44]) target: 53

context:tensor([44, 53]) target: 56

context:tensor([44, 53, 56]) target: 1

context:tensor([44, 53, 56,  1]) target: 58

context:tensor([44, 53, 56,  1, 58]) target: 46

context:tensor([44, 53, 56,  1, 58, 46]) target: 39

context:tensor([44, 53, 56,  1, 58, 46, 39]) target: 58

context:tensor([44, 53, 56,  1, 58, 46, 39, 58]) target: 1

context:tensor([52]) target: 58

context:tensor([52, 58]) target: 1

context:tensor([52, 58,  1]) target: 58

context:tensor([52, 58,  1, 58]) target: 46

context:tensor([52, 58,  1, 58, 46]) target: 39

context:tensor([52, 58,  1, 58, 46, 39]) target:

In [10]:
import torch.nn as nn
from torch.nn import functional as f

In [43]:
torch.manual_seed(1337)

class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding_table = nn.Embedding(vocab_size,vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.embedding_table(idx)
        if targets is None:
            loss = None
        else :
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = f.cross_entropy(logits,targets)
            
        return loss,logits

    def generate(self, idx, max_tokens):
        for _ in range(max_tokens):
            loss,logits = self(idx)
            logits = logits[:,-1,:]  
            probs = f.softmax(logits, dim = 1)          
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx,idx_next), dim = 1)
        return idx

model = BigramModel(vocab_size)
model = model.to(device)


loss,logits.shape
idx = torch.zeros((1,1), dtype = torch.long, device = device)
print(model.generate(idx,max_tokens=3))
print(decode(model.generate(idx,max_tokens=3)[0].tolist()))
        

tensor([[ 0, 31, 23, 21]])

JMT


In [44]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [48]:
batch_size = 32;

for _ in range(10000):
    x,y = get_batch('train')
    loss,logits = model(x,y)
    
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5881810188293457


In [85]:
torch.manual_seed(1337)
a,b,c = 4,8,2
x = torch.randn(a,b,c)
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [120]:
#brute force
res1 = torch.zeros(a,b,c)
for row in range(a):
    for col in range(b):
        prev = x[row,:col+1]
        mean = torch.mean(prev, dim = 0)
        res1[row,col] = mean
res1[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [121]:
# vectorized
tril = torch.tril(torch.ones(b,b))
t = tril/torch.sum(tril,dim = 1,keepdims = True)
res2  = t@x
res2[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [122]:
torch.allclose(res1,res2)

True

In [125]:
# alternative approach used in soft attention

z = torch.zeros((b,b))
z = z.masked_fill(tril == 0, float('-inf'))
soft_max = f.softmax(z,dim = 1)
res3 = soft_max @ x

torch.allclose(res2,res3)

True